# [TEAM 8] 프로젝트 1. 크롤링 및 시각화

## 1. 크롤링

### 1) 역할 분배


a. **도희** : 드라이브 오픈 - 네이버 열기 - 네이버 검색창 클릭 - 검색어 입력 - Keys.RETURN
        > 검색어 : ***펜션***, 펜션 추천, 가족 펜션, 키즈 펜션, 노키즈 펜션, 커플 펜션, 독채 펜션, 풀빌라 펜션, 스파 펜션, 복층 펜션, 오션뷰 펜션, 반려견 동반(애견?) 펜션, 신축 펜션)
        
b. **민재** : 인플루언서 탭 클릭 - if문 블로그 글 날짜(2021년 이후) 확인 - 인플루언서 팬수 및 글 url 가져오기

c. **옥영** : url 열기 - if 네이버 맵 태그 확인 - 블로그제목, 블로그 본문, 펜션이름, 위도, 경도  가져오기
    
* 크롤링 해온 데이터 저장 (dataframe)
    
    | 검색어 | 업로드날짜 | 블로그url | 인플루언서팬수 | 블로그제목 | 펜션이름 | 블로그본문 | 위도 | 경도 |
    | --- | --- | --- | --- | --- | --- | --- | --- | --- |
    |  |  |  |  |  |  |  |  |  |
    |  |  |  |  |  |  |  |  |  |

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta
import time
import pandas as pd
import numpy as np
import json
import re
import seaborn as sns
import matplotlib.pyplot as plt
import folium
import geopandas as gpd
from folium.plugins import MarkerCluster

C:\Users\dhfps\anaconda3\envs\geo\lib\site-packages\pyproj\__init__.py:91: UserWarning: Valid PROJ data directory not found. Either set the path using the environmental variable PROJ_LIB or with `pyproj.datadir.set_data_dir`.
  warnings.warn(str(err))


### 1) 도희

* 네이버 열고 검색어 입력 후 검색하기

In [2]:
search_word = input("검색어를 입력하세요:")

driver = webdriver.Chrome('./chromedriver')
driver.get("https://www.naver.com/")
driver.implicitly_wait(1)

elem = driver.find_element_by_class_name("input_text")

elem.send_keys(search_word)
elem.send_keys(Keys.RETURN)

driver.implicitly_wait(2)

검색어를 입력하세요:풀빌라 펜션


C:\Users\dhfps\AppData\Local\Temp/ipykernel_6736/741101771.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')
C:\Users\dhfps\AppData\Local\Temp/ipykernel_6736/741101771.py:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  elem = driver.find_element_by_class_name("input_text")


### 2) 민재

* 인플루언서 버튼 클릭

In [3]:
a = driver.find_element(By.CSS_SELECTOR, "li.menu:nth-child(5) > a")
b = driver.find_element(By.CSS_SELECTOR, "li.menu:nth-child(7) > a")

print(a.text)

if a.text == "인플루언서NEW":
    a.click()
else:
    b.click()
    
time.sleep(1)

인플루언서NEW


* 스크롤 제일 아래로 내리기

In [4]:
last_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:

    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

    time.sleep(2)

    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:    #last_height에 원하는 수치 입력
        break
    last_height = new_height
    

driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

* 인플루언서 탭에 올라와 있는 게시글 url, 업로드 날짜, 블로거 이름, 팬 수 크롤링

In [12]:
## 문자열을 날짜로 변환하는 함수

def str2date(date_str):

    if date_str[1:] == "일 전":    #일주일 이내 등록된 게시글
        date = datetime.today() - timedelta(days=int(date_str[0]))   #업로드 날짜 계산
        return date

    elif (date_str == "어제") or (date_str[2:] == "시간 전"):
        date = datetime.today() - timedelta(days=1)   #업로드 날짜 계산
        return date
    
    elif (date_str[1:] == "시간 전") or (date_str[1:] == "분전") or (date_str[2:] == "분전"):
        date = datetime.today()
        return date
    
    else:
        date_str = date_str.replace(".","-")[:-1]   #"%Y-%m-%d" 형식으로 변환
        date = datetime.strptime(date_str,"%Y-%m-%d")   #문자열을 날짜 형식으로 변환
        return date
    
    ## 함수 돌아가면 datetime 형식 날짜 리턴



### 크롤링 코드 ###

dates = []
bloger_names = []
fan_counts = []
urls = []

## 업로드 날짜
span_dates = driver.find_elements(By.CSS_SELECTOR, "div.detail_box._cross_trigger span.date")

for each in span_dates:
    date_str = each.text      #텍스트만 가져오기
    result_date = str2date(date_str)   #문자열인 date를 넣으면 datetime 형식으로 변환하는 함수 사용
    dates.append(result_date)

## 블로거 이름
names = driver.find_elements(By.CSS_SELECTOR, "div.user_box_inner a.name.elss")

for each in names:

    name = each.text
    bloger_names.append(name)
    
## 팬 수
fans = driver.find_elements(By.CSS_SELECTOR, "div.user_box_inner span._fan_count")

for each in fans:
    fan = each.text
    fan_counts.append(fan)

## 블로그 url
a_links = driver.find_elements(By.CSS_SELECTOR, "div.dsc_area > a.name_link")

for each in a_links:
    url = each.get_attribute("href")
    urls.append(url)


print(len(dates))
print(len(bloger_names))
print(len(fan_counts))
print(len(urls))

500
508
508
508


* (선택적) 날짜 이외의 정보들 개수 500개 초과시 아래 셀 실행

In [20]:
## 500개 초과된 변수가 존재한다면
## 앞쪽에서 부터 500개로 맞춰서 슬라이싱 후 데이터프레임 생성

if len(bloger_names) > 500:
    bloger_names = bloger_names[len(bloger_names)-500:]
    
if len(fan_counts) > 500:
    fan_counts = fan_counts[len(fan_counts)-500:]
    
if len(urls) > 500:
    urls = urls[len(urls)-500:]
    
    #개수 수정안해도 되도록 수정함(옥영)

* 팬 수 숫자로 수정하기

In [22]:
fans = []

for i in range(len(fan_counts)):
    count = re.sub("[만]?\+","000",fan_counts[i])
    count = int(re.sub("[.,]","",count))
    fans.append(count)

* 가져온 블로그 정보 데이터프레임으로 변환

In [23]:
search_blogs = pd.DataFrame()

search_blogs['dates'] = dates
search_blogs['bloger_names'] = bloger_names
search_blogs['fans'] = fans
search_blogs['urls'] = urls

search_blogs.head()

,dates,bloger_names,fans,urls
0,2021-12-27,열씨미,14000,https://in.naver.com/jbm993/contents/internal/...
1,2021-12-24,망고요정,13000,https://in.naver.com/mango/contents/internal/3...
2,2021-12-23,유에,1000,https://in.naver.com/rafiuta/contents/internal...
3,2021-12-02,일상탈출,13000,https://in.naver.com/tcacyc/contents/internal/...
4,2021-11-21,에릭샘,11000,https://in.naver.com/usgreen99/contents/intern...


* csv 파일로 저장

In [27]:
## search_blogs csv 파일로 저장
search_blogs.to_csv('search_blogs_pool.csv', header = False, index = False, encoding='euc-kr')

#검색어 확인 후 search_blogs_뒷부분 영어로 추가해주세요!!★

In [28]:
driver.close()   #크롬 닫기

## 3) 옥영

* 전체 코드

In [29]:
titles = []
main_contents = []
place_ids = []
place_names = []
place_addrs = []
place_tels = []
lats = []
lons = []
urls = []
blog_urls = []


## 크롬 숨겨서 크롤링하는 옵션
options = webdriver.ChromeOptions()
options.add_argument("headless")
options.add_argument("--no--sandbox")
options.add_argument("disable-dev-shm-usage")


### 크롤링 코드 ###

for url in range(len(search_blogs)):
    
    ## 블로그 url 열기
    driver = webdriver.Chrome('.\chromedriver', chrome_options = options)
    driver.get(search_blogs['urls'][url])
    time.sleep(1)
    
    ## 블로그인지 url 받아와서 확인
    current_url = driver.current_url
    
    if current_url[:23] == "https://blog.naver.com/":
    
        ## 본문 내용이 있는 iframe(mainFrame)으로 들어가기
        driver.switch_to.frame("mainFrame")
        
        ## map_info 있는지 확인
        check_elements = driver.find_elements(By.CSS_SELECTOR, "div.se-module > a.se-map-info")
        
        ## map_info 있으면 블로그 글 제목, 본문 내용, 펜션 이름, 위도, 경도 데이터 저장
        if len(check_elements) != 0:
            
            ## map_info에 네이버 맵 정보 가져오기
            map_info = driver.find_element(By.CSS_SELECTOR, "div.se-module > a.se-map-info")
            map_data = map_info.get_attribute("data-linkdata")
            map_dict = json.loads(map_data)   # str 형태의 map_data를 dict 형태로 변환
            time.sleep(1)

            ## 블로그 글 제목 가져오기
            title = driver.find_element(By.CSS_SELECTOR, "div.pcol1").text
            titles.append(title)
            print(title)

            ## 블로그 본문 내용 가져오기
            contents = driver.find_elements(By.CSS_SELECTOR, "div.se-component.se-text.se-l-default")
            texts = []
            for each in contents:
                texts.append(each.text)    #contents 안에 있는 text만 가져와서 저장
            text = " ".join(texts)    #list 요소로 저장된 문장들 하나로 이어붙이기
            main_contents.append(text)

            ## 펜션 정보 가져오기
            place_ids.append(map_dict['placeId'])
            place_names.append(map_dict['name'])
            place_addrs.append(map_dict['address'])
            place_tels.append(map_dict['tel'])
            lats.append(map_dict['latitude'])
            lons.append(map_dict['longitude'])
            
            ## url 저장하기
            url = search_blogs['urls'][url]
            urls.append(url)
            current_url = driver.current_url
            blog_urls.append(current_url)
            
            ## 웹 드라이버 종료
            driver.close()

        else:
            driver.close()
    else:
        driver.close()

C:\Users\dhfps\AppData\Local\Temp/ipykernel_6736/2837098447.py:22: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('.\chromedriver', chrome_options = options)
C:\Users\dhfps\AppData\Local\Temp/ipykernel_6736/2837098447.py:22: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('.\chromedriver', chrome_options = options)


경남 펜션 오션뷰 숙소 사천 풀빌라 독채 펜션
부산 펜션 송정해수욕장 풀빌라 호텔 후기
남해 숙소 추천 감성 오션뷰 수영장 노천탕 남해 풀빌라 펜션 웨이포인트풀빌라
경남 펜션 감성 넘치는 사천 풀빌라 독채펜션
대부도 풀빌라 디디모스 키즈 펜션 리얼 후기 
여수 독채 풀빌라 펜션 온수풀 수영장
국내 감성숙소 충북 제천 풀빌라 펜션 수페23 숙박 후기 
충청도 신상 숙소 독채 풀빌라 모음 ; 아산 스테이느느루, 청주 휘게, 공주 스테이오안, 옥천 시작에머물다, 신축 감성 펜션
영덕 펜션 하저스테이 :: 포항 근교 독채 풀빌라 가족여행
포항 풀빌라 펜션 하루 / 오션뷰 수영장, 스파 신축 숙소 ♥
남해 신축 감성숙소 디풀빌라 펜션 , 동남아 느낌 낭낭 🌴
경남 펜션 사천 풀빌라 펜션 시엘로리조트
울진숙소 추천 울진 풀빌라 펜션 산포리펜션에서 제대로 오션뷰펜션 
오션뷰 풀빌라 경주 펜션 추천 감포 숙소 샵앤플랫
전남 펜션 중 목포에서 가까운 무안 풀빌라 애모시옹
[여수여행] 여수 숙소 추천, 여수 블루망고 풀빌라 펜션
[가평숙소] 가평 풀빌라 펜션 온수풀 단독펜션 V528 리얼후기
거제도 펜션 프라이빗상상 풀빌라
경주 오푸스11 온수 풀파티 수영장 대가족 풀빌라펜션 추천 (겨울에 가볼만한곳 애견동반 스파)
가평 키즈펜션 수영장 풀빌라 펜션..
가평풀빌라 펜션 V528 가족 여행지로 추천해요! 
거제도 펜션 풀빌라 바다뷰는 덤~
여수 펜션 온수 풀빌라 리조트 슈가브리움
양평 숙소, 수영장 있는 커플 감성 풀빌라 펜션 그랑아치
제주도 풀빌라펜션 추천/ 오션브릭스, 산방산 아래의 아름다운 바다와 만나는 멋진 뷰
거제도 키즈 풀빌라펜션 모든 것이 만족에 만족
여수 독채 펜션 풀빌라 비고 리조트 황홀 오션뷰
강원도 춘천 유유자적 수영장 강뷰 풀빌라 펜션
경주 독채풀빌라 온수풀 펜션 스테이 마당 숙소 후기
거제도 숙소 흑진주마리나 리조트 풀빌라 펜션
거제 풀빌라 펜션 호텔상상 지심도 품은 오션뷰
순천 풀빌라 펜션 로얄맨션 감동 포인트!
경남 독채 풀빌라 레인보우 키즈 펜션 다녀

거제도 가족펜션 추천 레인보우키즈풀빌라펜션 바베큐파티 /곳간상점 (ft.일출명소)  
여수 키즈풀빌라 아기랑 갈만한 곳 추천, 자리움 풀빌라 독채 키즈펜션
가평 '리버하임' | 프라이빗한 독채 풀빌라에서 친구들과 파티
전라도 아기랑갈만한곳 여수 키즈풀빌라 펜션 추천
거제도펜션 거제 룰루랄라 키즈풀빌라 뷰도 좋아
가평 키즈펜션 추천 아기랑 루미나리에 풀빌라귀족펜션
아이와함께 어른들도 즐기자!!ㅋㅋㅋㅋ 가평 남이섬 풀빌라펜션 리버노스
경주 키즈 풀빌라 독채 펜션 마카롱 실내수영장과 마당을 갖춘 신축 펜션
담양 펜션 사계절 온수풀 즐기는 독채 풀빌라 더힐링나인
포항펜션 데이토나 풀빌라 역대급 오션뷰 숙소
대부도 키즈 풀빌라 펜션 포함 대부도 여행코스
전남 담양 애견동반풀빌라 구르미펜션 너무 행복했던 여행
여수키즈펜션 독채로 즐기는 유키즈풀빌라
경주키즈풀빌라 휴휴키즈펜션 feat. 주간생일
[대부도 블루블랑] 경기도 독채 키즈풀빌라 _ 서울근교 대형 온수수영장 펜션
온수수영장펜션 하늘호수키즈풀빌라 무료조식
노는언니2 폭포리아 풀빌라펜션에서 김장김치대결,김성연 챔피온로고후드티,신수지우승,한유미
남이섬 풀빌라 가평여행, 예쁜 복층인테리어 리버노스 가평 펜션
경주 풀빌라 독채 수영장 펜션 호플리
아쉬운 여수 오션뷰 숙소 : 블루망고 풀빌라 리조트 수영장 펜션
영덕 독채펜션 하저스테이
여수 숙소 추천 슈가브리움 리조트 여수 수영장 펜션 풀빌라
경주독채펜션 신축 풀빌라 누아르블랑 
경남 해돋이 명소 거제 펜션 7곳
경북가볼만한곳 ♥경북풀빌라,울진풀빌라 어디가좋을까 고민중  온수풀 실내수영장펜션인  풀빌라블루아라로 결정▶ 만족할만한 울진숙소 발견♪ 경북드라이브코스 찾아 다녀온 멋진 경북여행 
경주 키즈펜션 <키즈 플레이스> 풀빌라로 온전히 즐긴 하루
경주 키즈펜션 추천 루나힐풀빌라 에어바운스 룸
[포항숙소] 포항펜션 데이토나 풀빌라 오션뷰 포항풀빌라 이용후기
제주도 독채 풀빌라 노을정원 펜션
경주 펜션 엘라포니시 풀빌라 오션뷰 환상적
가평 키즈풀빌라 펜션 아이캐슬키즈풀빌라 아이

In [30]:
print(len(titles))
print(len(main_contents))
print(len(place_ids))
print(len(place_names))
print(len(place_addrs))
print(len(place_tels))
print(len(lats))
print(len(lons))
print(len(urls))
print(len(blog_urls))

442
442
442
442
442
442
442
442
442
442


* 크롤링한 데이터 데이터프레임으로 변환

In [31]:
blog_df = pd.DataFrame()

blog_df['titles'] = titles
blog_df['main_contents'] = main_contents
blog_df['place_ids'] = place_ids
blog_df['place_names'] = place_names
blog_df['place_addrs'] = place_addrs
blog_df['place_tels'] = place_tels
blog_df['lats'] = lats
blog_df['lons'] = lons
blog_df['urls'] = urls
blog_df['blog_urls'] = blog_urls

* search_blogs와 merge해서 최종 데이터 프레임 생성

In [32]:
result_df = pd.merge(blog_df, search_blogs, on='urls', how='left')
result_df.head()

,titles,main_contents,place_ids,place_names,place_addrs,place_tels,lats,lons,urls,blog_urls,dates,bloger_names,fans,year,month,day
0,경남 펜션 오션뷰 숙소 사천 풀빌라 독채 펜션,경남 펜션 오션뷰 숙소 사천 풀빌라 독채 펜션 펜캉스 힐링 경남 독채펜션~\n11월...,,사천리조트 풀빌라펜션,경상남도 사천시 토끼로 245 비토섬신우리조트,None,34.9844337,127.9663409,https://in.naver.com/jbm993/contents/internal/...,https://blog.naver.com/jbm993/222606263670?isI...,2021-12-27,열씨미,14000,2021,12,27
1,부산 펜션 송정해수욕장 풀빌라 호텔 후기,Busan - 온수풀 풀빌라 있는데 가성비 좋은 숙소 - 올해는 곳곳에 화이트 크리...,1102393804,그레이샌즈 풀빌라 펜션,부산광역시 해운대구 송정해변로 30-1 그레이샌즈 2층 3층 4층 5층,010-4074-0790,35.1809217,129.202743,https://in.naver.com/mango/contents/internal/3...,https://blog.naver.com/foresten/222603948009?i...,2021-12-24,망고요정,13000,2021,12,24
2,남해 숙소 추천 감성 오션뷰 수영장 노천탕 남해 풀빌라 펜션 웨이포인트풀빌라,제가 가장 좋아하는 여행지 중 한곳이 바로 남해에요!! 남해안 도로를 따라 드라이브...,1172720598,웨이포인트 풀빌라,경상남도 남해군 서면 남서대로 1803-64,010-8836-1388,34.818569,127.8266747,https://in.naver.com/rafiuta/contents/internal...,https://blog.naver.com/rafiuta/222602217417?is...,2021-12-23,유에,1000,2021,12,23
3,경남 펜션 감성 넘치는 사천 풀빌라 독채펜션,경남 펜션 사천 풀빌라\n토끼와 거북이의 이야기가 있는 사천 비토섬으로 2박 3일 ...,32779463,비토섬 신우리조트N풀빌라펜션,경상남도 사천시 서포면 토끼로 245-5,0507-1497-4244,34.9844333,127.9663639,https://in.naver.com/tcacyc/contents/internal/...,https://blog.naver.com/tcacyc/222584628097?isI...,2021-12-02,일상탈출,13000,2021,12,2
4,대부도 풀빌라 디디모스 키즈 펜션 리얼 후기,겨울이 코 앞으로 다가오면서 가족들을 위한 온수 풀장이 있는 펜션을 찾는 분들이 많...,1109160883,디디모스펜션,경기도 안산시 단원구 부흥로 287-292,010-5205-6393,37.2208471,126.6051739,https://in.naver.com/usgreen99/contents/intern...,https://blog.naver.com/usgreen99/222574303564?...,2021-11-21,에릭샘,11000,2021,11,21


* csv 파일로 저장

In [33]:
## pension 데이터 csv 파일로 저장
result_df.to_csv('pension_pool.csv', index = False, encoding='utf-8')

#검색어 확인하고 pension_뒷부분 바꿔주세요! ★

## 2. 데이터 정제

In [4]:
pension_couple = pd.read_csv("pension_couple.csv", encoding='utf-8')
pension_dog = pd.read_csv("pension_dog.csv", encoding='utf-8')
pension_family = pd.read_csv("pension_family.csv", encoding='utf-8')
pension_kids = pd.read_csv("pension_kids.csv", encoding='utf-8')
pension_nokids = pd.read_csv("pension_nokids.csv", encoding='utf-8')
pension_ocean = pd.read_csv("pension_ocean.csv", encoding='utf-8')
pension_pool = pd.read_csv("pension_pool.csv", encoding='utf-8')
pension_spa = pd.read_csv("pension_spa.csv", encoding='utf-8')

In [5]:
pension_couple['type'] = 'cpl'
pension_dog['type'] = 'dog'
pension_family['type'] = 'fmly'
pension_kids['type'] = 'kids'
pension_nokids['type'] = 'nokids'
pension_ocean['type'] = 'ocean'
pension_pool['type'] = 'pool'
pension_spa['type'] = 'spa'

In [6]:
print(pension_couple.shape)
print(pension_dog.shape)
print(pension_family.shape)
print(pension_kids.shape)
print(pension_nokids.shape)
print(pension_ocean.shape)
print(pension_pool.shape)
print(pension_spa.shape)

(362, 17)
(350, 17)
(436, 17)
(301, 17)
(89, 17)
(440, 17)
(442, 17)
(413, 17)


In [7]:
362+350+436+301+89+440+442+413

2833

In [8]:
pension_list = [pension_couple, pension_dog, pension_family, pension_kids, pension_nokids, pension_ocean, pension_pool, pension_spa]
pension_df = pd.concat(pension_list, ignore_index=True)

pension_df.shape

(2833, 17)

* year, month, day 컬럼 추가

In [11]:
pension_df['year'] = pension_df['dates'].dt.year
pension_df['month'] = pension_df['dates'].dt.month
pension_df['day'] = pension_df['dates'].dt.day

NameError: name 'blog_df' is not defined

In [9]:
pension_df['season'] = 'season'

pension_df['season'][pension_df['month']==12] = 'win'   #겨울
pension_df['season'][pension_df['month']==1] = 'win'
pension_df['season'][pension_df['month']==2] = 'win'
pension_df['season'][pension_df['month']==3] = 'spr'   #봄
pension_df['season'][pension_df['month']==4] = 'spr'
pension_df['season'][pension_df['month']==5] = 'spr'
pension_df['season'][pension_df['month']==6] = 'sum'   #여름
pension_df['season'][pension_df['month']==7] = 'sum'
pension_df['season'][pension_df['month']==8] = 'sum'
pension_df['season'][pension_df['month']==9] = 'aut'   #가을
pension_df['season'][pension_df['month']==10] = 'aut'
pension_df['season'][pension_df['month']==11] = 'aut'

pension_df['season'].head()

C:\Users\dhfps\AppData\Local\Temp/ipykernel_19940/2380678841.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pension_df['season'][pension_df['month']==12] = 'win'   #겨울
C:\Users\dhfps\AppData\Local\Temp/ipykernel_19940/2380678841.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pension_df['season'][pension_df['month']==1] = 'win'
C:\Users\dhfps\AppData\Local\Temp/ipykernel_19940/2380678841.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

0    win
1    aut
2    aut
3    win
4    win
Name: season, dtype: object

### 결측값 확인 및 처리

In [10]:
pension_df.info()     #place_ids, place_tels만 결측치 존재

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2833 entries, 0 to 2832
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   titles         2833 non-null   object 
 1   main_contents  2833 non-null   object 
 2   place_ids      2791 non-null   float64
 3   place_names    2833 non-null   object 
 4   place_addrs    2833 non-null   object 
 5   place_tels     2664 non-null   object 
 6   lats           2833 non-null   float64
 7   lons           2833 non-null   float64
 8   urls           2833 non-null   object 
 9   blog_urls      2833 non-null   object 
 10  dates          2833 non-null   object 
 11  bloger_names   2833 non-null   object 
 12  fans           2833 non-null   int64  
 13  year           2833 non-null   int64  
 14  month          2833 non-null   int64  
 15  day            2833 non-null   int64  
 16  type           2833 non-null   object 
 17  season         2833 non-null   object 
dtypes: float

In [34]:
pension_df = pension_df.fillna(0)     #결측치 모두 0으로 대체

In [35]:
pension_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2833 entries, 0 to 2832
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   titles         2833 non-null   object 
 1   main_contents  2833 non-null   object 
 2   place_ids      2833 non-null   float64
 3   place_names    2833 non-null   object 
 4   place_addrs    2833 non-null   object 
 5   place_tels     2833 non-null   object 
 6   lats           2833 non-null   float64
 7   lons           2833 non-null   float64
 8   urls           2833 non-null   object 
 9   blog_urls      2833 non-null   object 
 10  dates          2833 non-null   object 
 11  bloger_names   2833 non-null   object 
 12  fans           2833 non-null   int64  
 13  year           2833 non-null   int64  
 14  month          2833 non-null   int64  
 15  day            2833 non-null   int64  
 16  type           2833 non-null   object 
 17  season         2833 non-null   object 
dtypes: float

In [37]:
## pension_df csv 파일로 저장
pension_df.to_csv('pension_df.csv', index = False, encoding='utf-8')

In [2]:
## pension_df 파일 불러오기
pension_df = pd.read_csv("pension_df.csv", encoding='utf-8')

### 2021년만 사용

In [51]:
## 2021년 데이터만 슬라이싱

pension_2021 = pension_df[pension_df['year']==2021]
pension_2021.head()

,titles,main_contents,place_ids,place_names,place_addrs,place_tels,lats,lons,urls,blog_urls,dates,bloger_names,fans,year,month,day,type,season
0,"제주도 커플펜션 추천/ 제주N스테이, 바다가 보이는 예쁜 공간에서 하루를...",제주도 커플펜션 추천/ 제주N스테이 언제라도 갈대마다 마음이 편안해지고 즐거워지는 ...,1.467548e+09,제주N스테이,제주특별자치도 제주시 구좌읍 한동북1길 60 C동,0507-1324-0978,33.542778,126.830518,https://in.naver.com/gagamal740718/contents/in...,https://blog.naver.com/gagamal740718/222608098...,2021-12-30 16:46:59.995526,꼬미의 제주도 여행,1128,2021,12,30,cpl,win
1,제주 커플펜션 오션뷰 뷰스팟 제주도 서쪽 숙소,제주 커플펜션 오션뷰 뷰스팟 제주도 서쪽 숙소 제주도 서쪽 숙소 저바다에 누워~\n...,1.079993e+09,저바다에누워,제주특별자치도 제주시 한경면 판조로 3-5,0507-1440-7904,33.367228,126.205140,https://in.naver.com/jbm993/contents/internal/...,https://blog.naver.com/jbm993/222554609062?isI...,2021-10-31 00:00:00.000000,열씨미,14000,2021,10,31,cpl,aut
2,제주 오션뷰 멋진 제주도 커플 펜션 숙소,가을 제주가 너무 아름다워 아주 짧게 와이프와 제주도를 다시 다녀왔습니다. 1박 2...,1.347007e+09,저바다에누워,제주특별자치도 제주시 한경면 판조로 3-5,0,33.367236,126.205129,https://in.naver.com/ultraorange/contents/inte...,https://blog.naver.com/lightsout/222550520526?...,2021-10-27 00:00:00.000000,울트라오렌지,15000,2021,10,27,cpl,aut
3,경주 스파펜션 / 울산 스파펜션 <러브앤러브 펜션> #커플펜션 #내돈주고 다녀온 솔...,안녕하세요~\n오늘은 #경주오션힐스 펜션촌에 위치한\n#러브앤러브펜션 포스팅을 해볼...,3.080236e+07,러브앤러브,경상북도 경주시 양남면 솔숲3길 18,0503-5058-2619,35.664785,129.406206,https://in.naver.com/yt___26/contents/internal...,https://blog.naver.com/ysnjhfav/222585304055?i...,2021-12-02 00:00:00.000000,체로미,2011,2021,12,2,cpl,win
4,가평 커플 펜션 숙소 추천 라스블랑카스(LAS BLANCAS) 펜션 숙박 이용 리얼 후기,추억의 한자리를 차지하고 있는 가평 대성리 MT촌 펜션에서 하룻밤 숙박을 하고 왔습...,7.786633e+08,라스블랑카스,경기도 가평군 청평면 남이터길 54-12 라스블랑카스,010-8569-3340,37.689413,127.371586,https://in.naver.com/pilyo93/contents/internal...,https://blog.naver.com/pilyo93/222595781807?is...,2021-12-15 00:00:00.000000,맛집여행이정표,368,2021,12,15,cpl,win


### 팬 수별로 level col 생성

In [57]:
pension_2021['level'] = 0

for i in range(len(pension_2021)):
    
    if pension_2021['fans'][i] < 2500:
        pension_2021.loc[i,'level'] = 1
    elif pension_2021['fans'][i] < 5000:
        pension_2021.loc[i,'level'] = 2
    elif pension_2021['fans'][i] < 10000:
        pension_2021.loc[i,'level'] = 3
    else:
        pension_2021.loc[i,'level'] = 4

pension_2021['level'].head()

0    1
1    4
2    4
3    1
4    1
Name: level, dtype: int64

In [58]:
pension_2021['fans'].head()

0     1128
1    14000
2    15000
3     2011
4      368
Name: fans, dtype: int64

### 펜션 시도 col 생성

In [92]:
for i in range(len(pension_2021)):
    index = pension_2021.loc[i,'place_addrs'].find(" ")
    pension_2021.loc[i,'geo'] = pension_2021.loc[i,'place_addrs'][:index]
    
pension_2021.head()

,titles,main_contents,place_ids,place_names,place_addrs,place_tels,lats,lons,urls,blog_urls,dates,bloger_names,fans,year,month,day,type,season,level,geo
0,"제주도 커플펜션 추천/ 제주N스테이, 바다가 보이는 예쁜 공간에서 하루를...",제주도 커플펜션 추천/ 제주N스테이 언제라도 갈대마다 마음이 편안해지고 즐거워지는 ...,1.467548e+09,제주N스테이,제주특별자치도 제주시 구좌읍 한동북1길 60 C동,0507-1324-0978,33.542778,126.830518,https://in.naver.com/gagamal740718/contents/in...,https://blog.naver.com/gagamal740718/222608098...,2021-12-30 16:46:59.995526,꼬미의 제주도 여행,1128,2021,12,30,cpl,win,1,제주특별자치도
1,제주 커플펜션 오션뷰 뷰스팟 제주도 서쪽 숙소,제주 커플펜션 오션뷰 뷰스팟 제주도 서쪽 숙소 제주도 서쪽 숙소 저바다에 누워~\n...,1.079993e+09,저바다에누워,제주특별자치도 제주시 한경면 판조로 3-5,0507-1440-7904,33.367228,126.205140,https://in.naver.com/jbm993/contents/internal/...,https://blog.naver.com/jbm993/222554609062?isI...,2021-10-31 00:00:00.000000,열씨미,14000,2021,10,31,cpl,aut,4,제주특별자치도
2,제주 오션뷰 멋진 제주도 커플 펜션 숙소,가을 제주가 너무 아름다워 아주 짧게 와이프와 제주도를 다시 다녀왔습니다. 1박 2...,1.347007e+09,저바다에누워,제주특별자치도 제주시 한경면 판조로 3-5,0,33.367236,126.205129,https://in.naver.com/ultraorange/contents/inte...,https://blog.naver.com/lightsout/222550520526?...,2021-10-27 00:00:00.000000,울트라오렌지,15000,2021,10,27,cpl,aut,4,제주특별자치도
3,경주 스파펜션 / 울산 스파펜션 <러브앤러브 펜션> #커플펜션 #내돈주고 다녀온 솔...,안녕하세요~\n오늘은 #경주오션힐스 펜션촌에 위치한\n#러브앤러브펜션 포스팅을 해볼...,3.080236e+07,러브앤러브,경상북도 경주시 양남면 솔숲3길 18,0503-5058-2619,35.664785,129.406206,https://in.naver.com/yt___26/contents/internal...,https://blog.naver.com/ysnjhfav/222585304055?i...,2021-12-02 00:00:00.000000,체로미,2011,2021,12,2,cpl,win,1,경상북도
4,가평 커플 펜션 숙소 추천 라스블랑카스(LAS BLANCAS) 펜션 숙박 이용 리얼 후기,추억의 한자리를 차지하고 있는 가평 대성리 MT촌 펜션에서 하룻밤 숙박을 하고 왔습...,7.786633e+08,라스블랑카스,경기도 가평군 청평면 남이터길 54-12 라스블랑카스,010-8569-3340,37.689413,127.371586,https://in.naver.com/pilyo93/contents/internal...,https://blog.naver.com/pilyo93/222595781807?is...,2021-12-15 00:00:00.000000,맛집여행이정표,368,2021,12,15,cpl,win,1,경기도


In [59]:
## pension_2021 csv 파일로 저장
pension_2021.to_csv('pension_2021_new.csv', index = False, encoding='utf-8')

In [4]:
## pension_2021 파일 불러오기
pension_2021 = pd.read_csv("pension_2021.csv", encoding='utf-8')

## 시각화 위한 mini df 생성 

In [4]:
## 시도별 게시글 수

pension_div = pd.DataFrame(pension_2021['geo'].value_counts())
pension_div.reset_index(level=0, inplace=True)
pension_div.rename(columns = {'index' : 'ctp_name'}, inplace = True)
pension_div.rename(columns = {'geo' : 'post_count'}, inplace = True)
pension_div

,ctp_name,post_count
0,경기도,375
1,경상북도,342
2,경상남도,311
3,강원도,293
4,제주특별자치도,227
5,전라남도,153
6,충청남도,119
7,인천광역시,81
8,부산광역시,30
9,충청북도,27


In [5]:
## 월별 게시글 수

pension_month = pd.DataFrame(pension_2021['month'].value_counts())
pension_month.reset_index(level=0, inplace=True)
pension_month.rename(columns = {'month' : 'post_count'}, inplace = True)
pension_month.rename(columns = {'index' : 'month'}, inplace = True)
pension_month

,month,post_count
0,12,356
1,11,312
2,10,228
3,7,165
4,6,164
5,9,152
6,5,147
7,8,139
8,4,121
9,3,104


In [6]:
## 펜션 타입별 게시글 수

pension_type = pd.DataFrame(pension_2021['type'].value_counts())
pension_type.reset_index(level=0, inplace=True)
pension_type.rename(columns = {'type' : 'post_count'}, inplace = True)
pension_type.rename(columns = {'index' : 'type'}, inplace = True)
pension_type

,type,post_count
0,pool,371
1,ocean,334
2,fmly,304
3,spa,291
4,dog,257
5,kids,250
6,cpl,157
7,nokids,44


In [7]:
## 계절별 게시글 수

pension_season = pd.DataFrame(pension_2021['season'].value_counts())
pension_season.reset_index(level=0, inplace=True)
pension_season.rename(columns = {'season' : 'post_count'}, inplace = True)
pension_season.rename(columns = {'index' : 'season'}, inplace = True)
pension_season

,season,post_count
0,aut,692
1,win,476
2,sum,468
3,spr,372


In [8]:
## pension_div csv 파일로 저장
pension_div.to_csv('pension_div.csv', index = False, encoding='utf-8')

In [9]:
## pension_month csv 파일로 저장
pension_month.to_csv('pension_month.csv', index = False, encoding='utf-8')

In [10]:
## pension_type csv 파일로 저장
pension_type.to_csv('pension_type.csv', index = False, encoding='utf-8')

In [11]:
## pension_season csv 파일로 저장
pension_season.to_csv('pension_season.csv', index = False, encoding='utf-8')

In [6]:
## 월, 타입, 계절, 시도별 게시글 수 그룹 df 생성

group = pension_2021.groupby(["month","type","season","geo"])
p_group = group.size().reset_index(name='post_count')

p_group

,month,type,season,geo,post_count
0,1,cpl,win,강원도,1
1,1,cpl,win,경상남도,1
2,1,cpl,win,경상북도,1
3,1,cpl,win,인천광역시,1
4,1,cpl,win,제주특별자치도,1
...,...,...,...,...,...
612,12,spa,win,경상남도,5
613,12,spa,win,경상북도,4
614,12,spa,win,인천광역시,3
615,12,spa,win,전라남도,1


In [7]:
## p_group csv 파일로 저장
p_group.to_csv('p_group.csv', index = False, encoding='utf-8')